# Executing Job on Pasqal hardware

In [ ]:
import numpy as np
from threading import Thread

from qat.qpus import RemoteQPU

Simulation on Pasqal hardware and software can be performed using [Pulser](https://pulser.readthedocs.io/). This notebook first presents how to generate a `Job` that can be executed on Pasqal hardware from a Pulser `Sequence`, before showing how to execute it using Pasqal hardware. 

## Generating a Job to execute on Pasqal hardware

In [ ]:
from pulser import Pulse, Sequence
from pulser.waveforms import CustomWaveform
from pulser.devices import Device, AnalogDevice
from pulser_myqlm import IsingAQPU, FresnelQPU
from pulser_myqlm.myqlmtools import are_equivalent_schedules

On Pulser we can solve problems of shape 
$$ H = \hbar \sum_i \frac{\Omega(t)}{2}(\cos(\phi) \sigma_i^x + \sin(\phi) \sigma_i^y) - \frac{\delta(t)}{2}n_i + \frac{1}{2}\sum_{i\neq j}U_{ij}n_i n_j$$
with $\sigma_i^x$, $\sigma_i^y$, $\sigma_i^z$ the Pauli operators $X$, $Y$, $Z$ applied on qubit $i$ and $n_i = \frac{1-\sigma_i^z}{2}$.

The states associated with this Hamiltonian are the ground state $ \left|g\right> = (1, 0) $ and the rydberg state $ \left|r\right> = (0, 1) $ 

This Hamiltonian is named the <u>Ising Hamiltonian</u>. It is composed of a time-independent part, $\frac{1}{2}\sum_{i\neq j}U_{ij}n_i n_j$, and the rest of the terms that make a time-dependent Hamiltonian. In Pulser, this Hamiltonian is generated via a `Sequence` object. It is initialized by a `Device` and a `Register` that define the coefficients $U_{ij}$. Then, `Pulse`s are added to this `Sequence` to generate the time-dependent terms of this Hamiltonian. You can find more information about this Ising Hamiltonian in the [Pulser documentation](https://pulser.readthedocs.io/en/stable/review.html).  

The [pulser-myqlm package](https://github.com/pasqal-io/Pulser-myQLM) and its `IsingAQPU` class enables you to convert a Pulser Sequence into a MyQLM `Schedule` or `Job`. Let's start by generating a `Job` from a `Sequence`, before simulating it using Pasqal's hardware and software tools. 

### Converting a Pulser Sequence into a Schedule or Job

A Pulser `Sequence` starts by defining a `Device` and a `Register`.

Pulser provides examples of Pasqal devices in `pulser.devices`. Any of these devices can be used for simulation purposes, yet the device representing best Pasqal QPUs is `AnalogDevice`. 

In real pulser devices, only `Register` built from a `RegisterLayout` can be implemented. A `RegisterLayout` is a map of the traps to use to trap atoms and build a `Register` from. Some `RegisterLayout` are already calibrated. They can be found in the `calibrated_register_layouts` of the `Device`. The definition of a custom `RegisterLayout` (not from the calibrated `RegisterLayout`) is possible, but executing a `Sequence` with a custom `RegisterLayout` will take more time than executing a `Sequence` with a calibrated one. Let's have a look to one of the calibrated layouts of `AnalogDevice`:

In [ ]:
print(AnalogDevice.calibrated_register_layouts)
analog_layout = AnalogDevice.calibrated_register_layouts[
    "TriangularLatticeLayout(61, 5.0µm)"
]
analog_layout.draw()

To define a `Register` from a `RegisterLayout`, you must use the `define_register` method of the `RegisterLayout`. You can read more about registers and layouts [in the pulser documentation](https://pulser.readthedocs.io/en/stable/tutorials/reg_layouts.html).

Let's define a Register of triangular shape with sites 26, 35 and 30. The sites are spaced by 5µm.

In [ ]:
register = analog_layout.define_register(26, 35, 30)
register.draw()

Let's now build a `Sequence` with this `Register` and `AnalogDevice`. Pasqal QPUs and `AnalogDevice` supports only one channel, a `Rydberg.Global` channel. Pulses can be added to this channel to generate the Ising Hamiltonian described above. They are composed of a Rabi frequency $\Omega$ (in $rad/\mu s$), a detuning $\delta$ (in $rad/\mu s$) and a phase $\phi$ (in $rad$) and are defined every $ns$. 

In [ ]:
t0 = 100  # in ns
seq = Sequence(register, AnalogDevice)
seq.declare_channel("ryd_glob", "rydberg_global")
# no parametrized sequence for the moment
seq.add(
    Pulse(
        CustomWaveform([ti / 100 for ti in range(t0)]),  # in rad/µs
        CustomWaveform([(1 - ti) / 100 for ti in range(t0)]),  # in rad/µs
        0,  # in rad
    ),
    "ryd_glob",
)  # corresponds to H0
seq.add(Pulse.ConstantPulse(t0, 1, 0, 0), "ryd_glob")  # corresponds to H1
seq.add(Pulse.ConstantPulse(t0, 1, 0, np.pi / 2), "ryd_glob")  # corresponds to H2
seq.draw(mode="input", draw_phase_curve=True)

The class methods `IsingAQPU.convert_sequence_to_schedule` and `IsingAQPU.convert_sequence_to_job` performs the direct conversion from a `Sequence` to a `Schedule` or `Job`. One can verify that the created `Schedule` has the shape of the Ising Hamiltonian defined above, with time-dependent coefficients in front of local `X`, `Y`, `Z` gates applied on qubits 0, 1 and 2, and `ZZ` interactions between the qubits. The Ising Hamiltonian being defined in $rad/\mu s$, the time in the `Schedule` is defined in $\mu s$ for this schedule to be simulated in `AnalogQPU` or `QutipQPU`.  

In [ ]:
schedule_from_seq = IsingAQPU.convert_sequence_to_schedule(seq)
print(schedule_from_seq)

The pulses of the `Sequence` are the instructions sent to the hardware. If you want a model of the behaviour of the hardware, you should convert while setting the `modulation` argument to `True`. You can learn more about modulation [in the Pulser documentation](https://pulser.readthedocs.io/en/stable/tutorials/output_mod_eom.html).

In [ ]:
# Samples of the Sequence are in full line, modulated samples in hatched
# mod_schedule_from_seq contains the modulated samples of the analog_seq
seq.draw(draw_phase_curve=True)
mod_schedule_from_seq = IsingAQPU.convert_sequence_to_schedule(seq, modulation=True)
print(mod_schedule_from_seq)

It is also possible to convert the Pulser `Sequence` directly into a job using `IsingAQPU.convert_sequence_to_job`. You can define the number of shots of the job in the attribute `nbshots`. The default value asks for the maximum number of shots the device can take.

In [ ]:
job_from_seq = IsingAQPU.convert_sequence_to_job(seq, nbshots=1000, modulation=True)
print(job_from_seq)

## Simulating a Job on Pasqal hardware and simulation tools

Any `Job` implementing an Ising Hamiltonian can be simulated using a local or remote QPU of Qaptiva Access by using its `submit` method. The only condition on this QPU is that it must be able to simulate [Analog Jobs](https://myqlm.github.io/02_user_guide/01_write/02_analog_schedule/03_an_jobs.html).

A Job encapsulating a serialized Pulser `Sequence` under the key "abstr_dict" of the dictionary `Job.schedule._other` can be simulated using Pasqal hardware and simulation tools.

`IsingAQPU` has a `submit` method that can be used to simulate the sequence either using `pulser_simulation` or another qpu.

### Simulating a Job using pulser-simulation locally

Prior to submitting a `Job` to a `QPU`, it is good practice to test that its execution works well. With `pulser_simulation`, you can simulate a Pulser Sequence. More details can be found [here](https://pulser.readthedocs.io/en/stable/tutorials/simulating.html).

To simulate a MyQLM `Job` locally using `pulser-simulation`, you first have to create an `IsingAQPU` using its `from_sequence` method and by setting its qpu to `None`. Once the `IsingAQPU` created, the results are obtained by submitting it the `Job` to simulate:

In [ ]:
# Create an IsingAQPU
simulation_aqpu = IsingAQPU.from_sequence(seq, qpu=None)
# Simulate the sequence
# If job was converted from the sequence using modulation=True,
# modulated samples of the sequence are used for the simulation
result_from_pulser = simulation_aqpu.submit(job_from_seq)
print("Result is:", result_from_pulser, "\n")
print(
    r"Expressed as {state: probability}:",
    {sample.state: sample.probability for sample in result_from_pulser},
)

### Executing a Job on a FresnelQPU

The `FresnelQPU` class is a `QPUHandler` that interfaces a QPU. It should be instantiated on the same network as the QPU, using its public address. If the public address is set to `None`, the `FresnelQPU` uses pulser-simulation to simulate the `Job`:

In [ ]:
fresnel_qpu = FresnelQPU(base_uri=None, version="v1")
print("The QPU is operational: ", fresnel_qpu.is_operational)

To submit a MyQLM job to `FresnelQPU`, you need to furnish a serialized Pulser `Sequence` in `Job.schedule._other`. This `Sequence` must use a device compatible with the performances of the QPU. The device associated with the `FresnelQPU` is accessible via the `description` attribute of the `HardwareSpecs` object obtained with `get_specs`. `AnalogDevice` has the same main properties as this device, but can enable the use of pulses with higher amplitudes or detunings.

In [ ]:
# Get the HardwareSpecs of the QPU
specs = fresnel_qpu.get_specs()
print("QPU specs are:", specs, "\n")
# Device is stored in its serialized form in the description
# Deserialize it using Device.from_abstract_repr
device = Device.from_abstract_repr(specs.description)
print("Device used is", device, "\n")
result_from_qpu = fresnel_qpu.submit(job_from_seq)
print("Result is:", result_from_qpu, "\n")
print(
    r"Expressed as {state: probability}:",
    {sample.state: sample.probability for sample in result_from_qpu},
)

### Simulating a Job remotly on a FresnelQPU

To access the QPU remotly, it has to be deployed on a server. You can deploy `FresnelQPU` on a Qaptiva server via its `serve` method. Any user can then access this QPU via a `RemoteQPU`:

In [ ]:
# Deploy the QPU on a Qaptiva server
server_thread = Thread(target=fresnel_qpu.serve, args=(1234,))
server_thread.start()
# Access it remotly
qpu = RemoteQPU(1234, "localhost")

Users can simulate jobs remotly via the `submit` method of the `RemoteQPU` defined above or by defining a new QPU in `IsingAQPU` and submitting the job to it:

In [ ]:
simulation_aqpu.set_qpu(qpu)
result_from_remote = simulation_aqpu.submit(job_from_seq)
print("Result is:", result_from_remote, "\n")
print(
    r"Expressed as {state: probability}: ",
    {sample.state: sample.probability for sample in result_from_remote},
)